#### Ejemplo Log Likelihood
Consideramos un dataset de juguete como ejemplo, con solo dos variables, que tienen una clara dependencia:

In [62]:
A <- c(0,1,0,1,0,1,0,1)
B <- c(0,1,0,1,0,1,0,0)
data <- data.frame(A,B)
rownames(data) <- c("d1", "d2", "d3", "d4", "d5", "d6", "d7", "d8")
data

,A,B
,<dbl>,<dbl>
d1,0,0
d2,1,1
d3,0,0
d4,1,1
d5,0,0
d6,1,1
d7,0,0
d8,1,0


In [54]:
pA1 <- mean(A)
pA0 <- 1 - pA1
pB1 <- mean(B)
pB0 <- 1 - pB1
pB1_A1 <- mean(data[data$A == 1, "B" ])
pB0_A1 <- 1 - pB1_A1
pB1_A0 <- mean(data[data$A == 0, "B" ])
pB0_A0 <- 1 - pB1_A0

Claramente hay dependencia, ya que...

In [59]:
pB1 != pB1_A1
pB0 != pB0_A1

[1] TRUE

[1] TRUE

Consideremos dos modelos, $DAG_0$, que es el grafo vacío, es decir, A y B son independientes; y el modelo $DAG_1 = A \rightarrow B$. En el que B depende de A.

Calculemos el Log Likelihood de ambos. Queremos saber cuál es la probabilidad del dataset $\mathcal{D} = $ `data`. Como cada realización del dataset se considera independiente, tenemos que $ P(\mathcal{D}) = P(d_1)· P(d_2) · \ ... \ · P(d_8)$. Es decir, y dado que el logaritmo es una función monótona creciente:

$$log(P(\mathcal{D})) = \sum_{j =1}^8 log(P(d_j))$$

Lo que queremos saber es qué grafo $DAG$ maximiza $log(P(\mathcal{D} \ | \ DAG))$.

Calculémoslo para ambos DAG.

**Para el grafo vacío...**

La factorización que tenemos es $P(A,B) = P(A)P(B)$.

- $P(d_1) = P(d_3) = P (d_5) = P(d_7) = P(A = 0)P(B = 0) = 0.3125$
- $P(d_2) = P(d_4) = P (d_6) = P(A = 1)P(B = 1) = 0.1875$
- $P(d_8) = P(A = 1)P(B = 0) = 0.3125$

In [69]:
p0 <- c(pA0*pB0,
        pA1*pB1,
        pA1*pB0)
p0

[1] 0.3125 0.1875 0.3125

**Para el grafo $A \rightarrow B$ ...**

La factorización que tenemos es $P(A,B) = P(A)P(B \ | \ A)$.

- $P(d_1) = P(d_3) = P (d_5) = P(d_7) = P(A = 0)P(B = 0 \ | \ A = 0) = 0.5$
- $P(d_2) = P(d_4) = P (d_6) = P(A = 1)P(B = 1 \ | \ A = 1) = 0.375$
- $P(d_8) = P(A = 1)P(B = 0 \ | \ A = 1) = 0.125$

In [71]:
p1 <- c(pB0_A0*pA0,
        pB1_A1*pA1,
        pB0_A1*pA1)
p1

[1] 0.500 0.375 0.125

**¿Cuál tiene mayor Log Likelihood?**

$log(P(\mathcal{D} \ | \ DAG_0)) = $

In [73]:
4*log(p0)[1] + 3*log(p0)[2] + log(p0)[3]

[1] -10.83768

$log(P(\mathcal{D} \ | \ DAG_1)) = $

In [74]:
4*log(p1)[1] + 3*log(p1)[2] + log(p1)[3]

[1] -7.794518

**Comprobación con `bnlearn`**:

In [77]:
dag0 <- empty.graph(c("A", "B"))
dag1 <- empty.graph(c("A", "B"))
dag1 <- set.arc(dag1, "A", "B")

In [84]:
data$A <- as.factor(data$A)
data$B <- as.factor(data$B)

bnlearn::score(dag0, data = data, type = "loglik")
bnlearn::score(dag1, data = data, type = "loglik")

[1] -10.83768

[1] -7.794518